<a href="https://colab.research.google.com/github/Elwing-Chou/ml0602/blob/main/sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
    extract=True,
)

In [14]:
import os
import glob
dn = os.path.split(dataset)[0]
fn = os.path.join(dn, "aclImdb", "train", "pos", "*")
print(fn)
fns = glob.glob(fn)
with open(fns[0], "r", encoding="utf-8") as f:
    print(f.read())

/root/.keras/datasets/aclImdb/train/pos/*
I was told it was one of those "either you love it or you hate it" movies. Well, I loved it. Obvious hippie-era, dated and easy symbolism and all. So, I probably have no taste at all when it comes to Antonioni, but this and La Notte (made exactly a decade earlier) are my favourites among his movies so far. Made two years before I was born, Zabriskie Point was supposed to have been Michelangelo's great American epic. But apparently, it turned out to be a flop. I really can't see why. Before watching it I'd read that it was rather boring, so I braced myself for a very slow movie - though I love me a slow movie. For my taste, Zabriskie didn't have a tedious minute in it. While watching it, I made a mental note of how European it was on the director's part to make such frequent use of advertisement billboards in almost every urban scene, enormous billboards dwarfing any human form in sight. This recurrent visual element is obviously there to underl

In [15]:
import pandas as pd
def get_data(base="train"):
    contents, target = [], []
    dn = os.path.split(dataset)[0]
    fn = os.path.join(dn, "aclImdb", base, "pos", "*.txt")
    fns = glob.glob(fn)
    for p in fns:
        with open(p, "r", encoding="utf-8") as f:
            contents.append(f.read())
            target.append(1)
    fn = os.path.join(dn, "aclImdb", base, "neg", "*.txt")
    fns = glob.glob(fn)
    for p in fns:
        with open(p, "r", encoding="utf-8") as f:
            contents.append(f.read())
            target.append(0)
    df = pd.DataFrame({
        "contents":contents,
        "ans":target
    })
    return df
train_df = get_data(base="train")
test_df = get_data(base="test")

In [21]:
# Step1. Tokenize
# 我們絕對不會使用0, 因為0是拿來做padding
from tensorflow.keras.preprocessing.text import Tokenizer
tok = Tokenizer(num_words=3000)
tok.fit_on_texts(train_df["contents"])

In [ ]:
# Step2. Sequenize
# tok.word_index
# tok.index_word[3000]
# 文章(token) -> sequence
x_train_seq = tok.texts_to_sequences(train_df["contents"])
x_test_seq = tok.texts_to_sequences(test_df["contents"])
pd.DataFrame(x_train_seq)

In [35]:
SEQ_LEN = 512

In [36]:
# Step3. Padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train_pad = pad_sequences(x_train_seq, maxlen=SEQ_LEN)
x_test_pad = pad_sequences(x_test_seq, maxlen=SEQ_LEN)
pd.DataFrame(x_train_pad)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,123,811,14,227,14,91,268,18,10,188,64,86,256,67,91,932,2,386,278,4,916,763,112,44,32,605,35,49,2,35,9,183,5,27,32,12,657,244,71,2338
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,64,620,31,1,1867,2,56,183,37,3,1,2120,133,6,2,160,577,44,54,577,16,12,191,2120,2,24,2161,16,1,269,1347,11,6,46,84,250,15,1,223,220
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,938,22,205,5,1,127,10,37,131,2567,4,99,2,10,431,101,11,28,889,1,1270,7,7,5,69,9,13,287,146,50,71,277,35,634,199,9,32,706,20,894
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,120,175,1,115,56,67,78,10,63,506,12,19,2,293,16,1,169,339,3,244,354,751,2,467,3,73,50,218,751,71,339,48,3,2233,9,1478,615,243,20,285
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,123,1244,3,2211,90,8,1,227,2916,11,141,199,22,139,5,101,41,7,7,1335,2,542,1174,356,5,64,50,37,11,5,29,4,145,571,8,1,228,4,11,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,11,44,32,218,640,917,922,111,18,42,1050,16,336,455,2,315,299,6,2270,464,1,580,4,3,864,62,2,455,16,38,673
24996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,4,2312,39,1,52,854,236,36,2688,1,694,174,583,2330,2,1839,2546,78,114,5,335,7,7,47,63,215,3,173,5,132,40,199,9,3,715,7,7,2712,2433,434
24997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,430,339,378,15,49,225,1,1139,13,63,2697,10,216,9,20,2666,10,158,987,49,275,5,64,9,30,3,747,143,128,3,114,2416,30,86,108,84,853,11,17,44
24998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,766,47,13,35,114,39,57,133,940,12,9,59,27,875,15,256,21,1077,16,1,62,57,5,791,1,191,1386,13,12,1,451,35,73,437,2,92,1,127,9


In [37]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense
layers = [
    # 每個詞彙都有128個係數, 3001 * 128 = 384128
    Embedding(3001, 128, mask_zero=True, input_length=SEQ_LEN),
    GlobalAveragePooling1D(),
    Dense(2, activation="softmax")
]
model = Sequential(layers)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 512, 128)          384128    
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 258       
Total params: 384,386
Trainable params: 384,386
Non-trainable params: 0
_________________________________________________________________
